# Data analysis with Polars
**_Use of Polars to analyze the NYC taxi dataset and the Amazon Reviews dataset_**

This notebook works well with the `conda_python3` kernel on a SageMaker Notebook `ml.t3.2xlarge` instance.

---
---

## Contents

1. [Objective](#Objective)
1. [NYC Taxi dataset](#NYC-taxi-dataset)
1. [Amazon Reviews dataset](#Amazon-reviews-dataset)
1. [Conclusion](#Conclusion)

---

## Objective

1. Understand how to use Polars for the typical operations such as:
    - Reading data from S3.
    - Group by and compute aggregates.
    - Combine multiple operations into an expression that is evaluate lazily.
    - Use Polars for working with text fields in a dataframe.
    
1. Do a side by side comparison of Polars with Pandas 2.0.
    

---

## Setup

Install the `pandas` 2.0 and polars packages that we will be using in this notebook.

In [1]:
!pip install polars==0.17.15  pandas==2.0.0 pyarrow==12.0.0

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 38.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 59.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 18.5 MB/s eta 0:00:0000:0100:01
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.2
    Uninstalling pandas-1.5.2:
      Successfully uninstalled pandas-1.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.20.3 requires nest-asyncio==1.5.5, but you have nest-asyncio 1.5.6 which is incompatible.


In [4]:
import pandas as pd
import polars as pl

In [5]:
print(f"pandas={pd.__version__}, polars={pl.__version__}")

pandas=2.0.0, polars=0.17.15


---

## NYC taxi dataset

The NYC taxi data is available as public data in the `s3://nyc-tlc` bucket. We read this data in Pandas first and then in Polars and then do operations on this data in Pandas and Polars side by side to compare performance.

### Read the data

The data consists of ~40 million rows. **Polars is able to read data about 35% faster than Pandas**.

In [6]:
%%time
df_list = []
for i in range(12):
    df = pd.read_parquet(f"s3://nyc-tlc/trip data/yellow_tripdata_2022-{i+1:02d}.parquet")
    df_list.append(df)
df_pd = pd.concat(df_list)

CPU times: user 22.4 s, sys: 11.7 s, total: 34 s
Wall time: 20.7 s


In [7]:
print(f"df_pd shape={df_pd.shape}")
display(df_pd.head())

df_pd shape=(39656098, 19)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [8]:
%%time
df_list = []
for i in range(12):
    df = pl.read_parquet(f"s3://nyc-tlc/trip data/yellow_tripdata_2022-{i+1:02d}.parquet")
    df_list.append(df)
df_pl = pl.concat(df_list)

CPU times: user 21.9 s, sys: 8.12 s, total: 30 s
Wall time: 14.8 s


In [9]:
%%time
print(f"df_pl shape={df_pl.select(pl.count())[0,0]}x{df_pl.width}")
display(df_pl.head())

df_pl shape=39656098x19


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
i64,datetime[ns],datetime[ns],f64,f64,f64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.8,1.0,"""N""",142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.1,1.0,"""N""",236,42,1,8.0,0.5,0.5,4.0,0.0,0.3,13.3,0.0,0.0
2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,"""N""",166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,"""N""",114,68,2,8.0,0.5,0.5,0.0,0.0,0.3,11.8,2.5,0.0
2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.3,1.0,"""N""",68,163,1,23.5,0.5,0.5,3.0,0.0,0.3,30.3,2.5,0.0


CPU times: user 5.48 ms, sys: 2.4 ms, total: 7.88 ms
Wall time: 35.7 ms


### Count and schema

We convert the datafrmae in to a `LazyFrame` and run a count much like we would do it in a SQL select statement.

In [10]:
%%time
q = (
        df_pl
        .lazy()
        .select(
            pl.count()
    )    
)
q.collect()

CPU times: user 1.6 ms, sys: 338 µs, total: 1.94 ms
Wall time: 3.03 ms


count
u32
39656098


In [11]:
df.schema

{'VendorID': Int64,
 'tpep_pickup_datetime': Datetime(time_unit='ns', time_zone=None),
 'tpep_dropoff_datetime': Datetime(time_unit='ns', time_zone=None),
 'passenger_count': Float64,
 'trip_distance': Float64,
 'RatecodeID': Float64,
 'store_and_fwd_flag': Utf8,
 'PULocationID': Int64,
 'DOLocationID': Int64,
 'payment_type': Int64,
 'fare_amount': Float64,
 'extra': Float64,
 'mta_tax': Float64,
 'tip_amount': Float64,
 'tolls_amount': Float64,
 'improvement_surcharge': Float64,
 'total_amount': Float64,
 'congestion_surcharge': Float64,
 'airport_fee': Float64}

### Group by and aggregate

Again, **Polars is about 60% faster when doing a group by on two columns, count within the groups and then sort**.

In [12]:
%%time
df_pd.groupby(["PULocationID", "DOLocationID"]).size().sort_values(ascending=False).reset_index(name='count')

CPU times: user 1.47 s, sys: 607 ms, total: 2.08 s
Wall time: 2.09 s


,PULocationID,DOLocationID,count
0,237,236,258785
1,236,237,219336
2,264,264,202065
3,237,237,174651
4,236,236,166264
...,...,...,...
43896,119,195,1
43897,119,192,1
43898,119,188,1
43899,119,183,1


In [13]:
%%time

q = (
        df_pl
        .lazy()
        .groupby(["PULocationID", "DOLocationID"])
        .agg(
        [
            pl.count()
        ]
    )
    .sort(["count"], descending=True)
)
q.collect()

CPU times: user 3.78 s, sys: 359 ms, total: 4.14 s
Wall time: 676 ms


PULocationID,DOLocationID,count
i64,i64,u32
237,236,258785
236,237,219336
264,264,202065
237,237,174651
236,236,166264
237,161,113630
161,237,103160
132,132,101095
142,239,98503


### Combine multiple operations in a single expression

This allows Polars to run the query optimizer and really provide the benefits in terms of reduced execution time. Plus the Polars code is more expressive code, somewhat similar to `tidyverse` in R. 

The Pandas operations, even though vectorized, still need to be done one after the other and cannot be combined into a pipeline.

***The Polars version of the following code about 5 to 6 times faster than Pandas code!!!***

In [14]:
%%time
df_pd['tpep_pickup_datetime_year'] = df_pd.tpep_pickup_datetime.dt.year
df_pd['tpep_pickup_datetime_month'] = df_pd.tpep_pickup_datetime.dt.month
df_pd['tpep_pickup_datetime_day'] = df_pd.tpep_pickup_datetime.dt.day

CPU times: user 9.14 s, sys: 162 ms, total: 9.3 s
Wall time: 9.3 s


In [15]:
%%time
df_pd = df_pd[df_pd.tpep_pickup_datetime_year == 2022]

CPU times: user 2.7 s, sys: 1.46 s, total: 4.16 s
Wall time: 4.06 s


In [16]:
%%time
df_pd_grouped_stats = df_pd.groupby(["tpep_pickup_datetime_year", "tpep_pickup_datetime_month", "tpep_pickup_datetime_day"]) \
                           .aggregate({'airport_fee': 'size', 'trip_distance': 'mean', 'fare_amount': 'mean',
                                       'mta_tax': 'mean', 'tip_amount': 'mean',
                                       'passenger_count': 'mean', 'total_amount': 'mean'}) \
                           .rename({'airport_fee': 'count'}, axis=1) \
                           .sort_values(by=["tpep_pickup_datetime_year", "tpep_pickup_datetime_month", "tpep_pickup_datetime_day"], ascending=False) \
                           .reset_index()
df_pd_grouped_stats

CPU times: user 3.38 s, sys: 758 ms, total: 4.14 s
Wall time: 4.14 s


,tpep_pickup_datetime_year,tpep_pickup_datetime_month,tpep_pickup_datetime_day,count,trip_distance,fare_amount,mta_tax,tip_amount,passenger_count,total_amount
0,2022,12,31,85075,5.496663,18.808202,0.484683,3.162040,1.522136,26.715433
1,2022,12,30,92429,3.934692,20.768060,0.484105,3.461874,1.507088,29.836820
2,2022,12,29,93498,3.791165,20.579808,0.486399,3.429100,1.514585,29.533789
3,2022,12,28,89065,4.193342,20.609701,0.487004,3.428778,1.510301,29.564857
4,2022,12,27,86290,4.205887,20.883973,0.485821,3.418239,1.514512,29.909588
...,...,...,...,...,...,...,...,...,...,...
360,2022,1,5,74592,5.916514,13.105423,0.491286,2.318954,1.372645,19.345578
361,2022,1,4,74562,3.472559,13.474487,0.490814,2.357847,1.384201,19.797229
362,2022,1,3,72405,7.703310,14.345576,0.488986,2.486244,1.408575,20.908380
363,2022,1,2,58421,6.210313,16.001280,0.486851,2.765876,1.501177,22.756311


In [17]:
%%time
q = (
        df_pl
        .lazy()
        .with_columns(pl.col("tpep_pickup_datetime").dt.year().suffix("_year"))
        .with_columns(pl.col("tpep_pickup_datetime").dt.month().suffix("_month"))
        .with_columns(pl.col("tpep_pickup_datetime").dt.day().alias("tpep_pickup_datetime_day"))
        .filter(pl.col("tpep_pickup_datetime_year") == 2022)
        .groupby(["tpep_pickup_datetime_year", "tpep_pickup_datetime_month", "tpep_pickup_datetime_day"])
        .agg(
        [
            pl.count(),
            pl.col("trip_distance").mean().prefix("mean_"),
            pl.col("fare_amount").mean().prefix("mean_"),
            pl.col("mta_tax").mean().prefix("mean_"),
            pl.col("tip_amount").mean().prefix("mean_"),
            pl.col("passenger_count").mean().prefix("mean_"),
            pl.col("total_amount").mean().prefix("mean_"),
        ]
    )
    .sort(["tpep_pickup_datetime_year", "tpep_pickup_datetime_month", "tpep_pickup_datetime_day"], descending=True)
)
df_pl_grouped_stats = q.collect()
df_pl_grouped_stats

CPU times: user 4.73 s, sys: 2.04 s, total: 6.78 s
Wall time: 3.22 s


tpep_pickup_datetime_year,tpep_pickup_datetime_month,tpep_pickup_datetime_day,count,mean_trip_distance,mean_fare_amount,mean_mta_tax,mean_tip_amount,mean_passenger_count,mean_total_amount
i32,u32,u32,u32,f64,f64,f64,f64,f64,f64
2022,12,31,85075,5.496663,18.808202,0.484683,3.16204,1.522136,26.715433
2022,12,30,92429,3.934692,20.76806,0.484105,3.461874,1.507088,29.83682
2022,12,29,93498,3.791165,20.579808,0.486399,3.4291,1.514585,29.533789
2022,12,28,89065,4.193342,20.609701,0.487004,3.428778,1.510301,29.564857
2022,12,27,86290,4.205887,20.883973,0.485821,3.418239,1.514512,29.909588
2022,12,26,67459,4.130195,21.092792,0.483694,3.341734,1.564689,29.594587
2022,12,25,50109,4.138615,20.571393,0.481532,3.150251,1.55646,28.624608
2022,12,24,75519,3.329849,17.61407,0.483615,2.835761,1.514524,25.025802
2022,12,23,95028,3.455618,18.478721,0.485021,3.123522,1.449239,26.942867


---

## Amazon reviews dataset


In [18]:
!aws s3 ls s3://amazon-reviews-pds/parquet/product_category=Books/

2018-04-09 06:35:58 1094842361 part-00000-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet
2018-04-09 06:35:59 1093295804 part-00001-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet
2018-04-09 06:36:00 1095643518 part-00002-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet
2018-04-09 06:36:00 1095218865 part-00003-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet
2018-04-09 06:36:00 1094787237 part-00004-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet
2018-04-09 06:36:33 1094302491 part-00005-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet
2018-04-09 06:36:35 1094565655 part-00006-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet
2018-04-09 06:36:35 1095288096 part-00007-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet
2018-04-09 06:36:35 1092058864 part-00008-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet
2018-04-09 06:36:35 1093613569 part-00009-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet


In [20]:
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
stopwords.append(["&", "\"\""])
stopwords[:10]

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

This is a huge dataset for a `t3.2xlarge` intance so we will just read one file.

In [22]:
%%time
import polars as pl
df_list = []
for i in range(1):
    df = pl.read_parquet(f"s3://amazon-reviews-pds/parquet/product_category=Books/part-0000{i}-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet")
    df_list.append(df)
df_pl_reviews = pl.concat(df_list)

CPU times: user 8.12 s, sys: 5.75 s, total: 13.9 s
Wall time: 23.2 s


Shape of the dataframe, more than 2 million rows containing text data and 15 columns per row.

In [23]:
%%time
q = (
        df_pl_reviews
        .lazy()
        .select(
            pl.count()
    )    
)
count = q.collect()[0,0]

print(f"df_pl_reviews shape={count}x{df_pl_reviews.width}")

df_pl_reviews shape=2074682x15
CPU times: user 1.09 ms, sys: 0 ns, total: 1.09 ms
Wall time: 738 µs


In [24]:
display(df_pl_reviews.head())

marketplace,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,year
str,str,str,str,str,str,i32,i32,i32,str,str,str,str,date,i32
"""US""","""15444933""","""R1WWG70WK9VUCH…","""1848192576""","""835940987""","""Standing Qigon…",5,9,10,"""N""","""Y""","""Informative AN…","""After attendin…",2015-05-02,2015
"""US""","""20595117""","""R1EQ3POS0RIOD5…","""145162445X""","""574044348""","""A Universe fro…",4,4,7,"""N""","""N""","""Between 'Nothi…","""Krauss traces …",2012-06-29,2012
"""US""","""52925878""","""R10SRJA4VVGUBD…","""055341805X""","""846590203""","""Hyacinth Girls…",4,0,0,"""Y""","""N""","""Mysteries upon…","""Rebecca, a den…",2015-05-02,2015
"""US""","""40062567""","""RD3268X41GM7U""","""0425263908""","""119148606""","""Bared to You""",5,1,1,"""N""","""N""","""""RAW, STEAMY, …","""\\""BARED TO YO…",2012-06-29,2012
"""US""","""47221489""","""R3KGQL5X5BSJE1…","""1416556141""","""987400385""","""Healer: A Nove…",5,0,0,"""N""","""Y""","""Well written s…","""Good character…",2015-05-02,2015


### Regex based filtering

Lets filter for all the rows of data that contain some words of interest. We use the `product_title` and `review_body` fields for this. We then tokenized these fields in the filtered dataframe to find out the number of words in these fields.

In [25]:
%%time
import re
topic_regex = r"(?i)nature|health|fitness|consciousness|diet|exercise"
q = (
        df_pl_reviews
        .lazy()
        .filter(
          pl.col('product_title').str.contains(topic_regex) |
            pl.col('review_body').str.contains(topic_regex)
        )
        .with_columns(pl.col("product_title").str.split(by=" ").suffix('_tokens'))
        .with_columns(pl.col("review_body").str.split(by=" ").suffix('_tokens'))
        .select(
            pl.col("review_date"),
            pl.col("year"),
            pl.col("product_title"),
            pl.col("product_title_tokens"),            
            pl.col("product_title_tokens").arr.lengths().suffix("_length"),            
            pl.col("review_body"),
            pl.col("review_body_tokens"),
            pl.col("review_body_tokens").arr.lengths().suffix("_length"),
            pl.col("star_rating"),
            pl.col("total_votes")
        )
    )
df_result = q.collect()
print(f"shape of result dataframe = {df_result.shape}")
df_result.head()

shape of result dataframe = (133784, 10)
CPU times: user 3.52 s, sys: 833 ms, total: 4.35 s
Wall time: 3.73 s


review_date,year,product_title,product_title_tokens,product_title_tokens_length,review_body,review_body_tokens,review_body_tokens_length,star_rating,total_votes
date,i32,str,list[str],u32,str,list[str],u32,i32,i32
2015-05-02,2015,"""Standing Qigon…","[""Standing"", ""Qigong"", … ""Zhuang""]",10,"""After attendin…","[""After"", ""attending"", … ""it.""]",159,5,10
2015-05-02,2015,"""Hyacinth Girls…","[""Hyacinth"", ""Girls:"", … ""Novel""]",4,"""Rebecca, a den…","[""Rebecca,"", ""a"", … ""Girls.&#34;""]",316,4,0
2012-06-29,2012,"""The Missionary…","[""The"", ""Missionary"", … ""Practice""]",9,"""This book was …","[""This"", ""book"", … ""section.""]",548,4,7
2012-06-29,2012,"""Wheat Belly: L…","[""Wheat"", ""Belly:"", … ""Health""]",15,"""This is an exc…","[""This"", ""is"", … ""off!""]",125,5,1
2012-06-29,2012,"""Consciousness …","[""Consciousness"", ""Beyond"", … ""Experience""]",9,"""Consciousness …","[""Consciousness"", ""Beyond"", … ""Spirit""]",506,5,0


Now we exclude the stopwords (using the `stopwords` list from NLTK) and then create a count of words using the tokenized `product_title_token` field. We use the `explode` function to convert the token list into a rows.

In [26]:
%%time
q = (
    df_pl_reviews
    .sample(fraction=1)
    .lazy()        
    .filter(
          pl.col('product_title').str.contains(topic_regex) |
            pl.col('review_body').str.contains(topic_regex)
    )    
    .with_columns(pl.col("product_title").str.split(by=" ").suffix('_tokens'))
    .explode("product_title_tokens")
    .with_columns(pl.col("product_title_tokens").str.to_lowercase())
    .filter(pl.col("product_title_tokens").is_in(stopwords).is_not())
    .groupby(["product_title_tokens"])
        .agg(
         [
            pl.count(),            
         ]
        )
         .sort(["count"], descending=True)
)
q.collect()

CPU times: user 8.93 s, sys: 12.9 s, total: 21.8 s
Wall time: 9.68 s


product_title_tokens,count
str,u32
"""health""",11466
"""guide""",9649
"""healthy""",6632
"""life""",6230
"""&""",6226
"""diet""",5672
"""lose""",5369
"""recipes""",5348
"""weight""",5183


---

## Conclusion

Polars is still faster than Pandas 2.0 and allows us to write expressive code that can be optimized.